In [3]:
import pandas as pd
import json
import os
import re

In [51]:
def get_character_index(story_lines):
  indexes = []
  cur_index = 0
  for i in range(len(story_lines)):
    indexes.append(cur_index)
    cur_index += len(story_lines[i])
  indexes.append(cur_index)
  return indexes

In [52]:
schoolmistress_story_paragraphs = open('schoolmistress_paragraphs.txt', 'r').readlines()
schoolmistress_story_paragraphs_indexes = get_character_index(schoolmistress_story_paragraphs)
schoolmistress_story_sentences = open('schoolmistress.txt', 'r').readlines()
schoolmistress_story_sentences_indexes = get_character_index(schoolmistress_story_sentences)

el_story_paragraphs = open('expensivelessons_paragraphs.txt', 'r').readlines()
el_story_paragraphs_indexes = get_character_index(el_story_paragraphs)
el_story_sentences = open('expensivelessons.txt', 'r').readlines()
el_story_sentences_indexes = get_character_index(el_story_sentences)

In [53]:
def highlight_percent():
  return ''

In [54]:
def update_indexes(highlight, indexes):
  for i in range(len(highlight) - 1):
    highlight[i]['start'] = highlight[i]['start'] + indexes[highlight[-1]['paragraph']]
    highlight[i]['end'] = highlight[i]['end'] + indexes[highlight[-1]['paragraph']]
  return highlight

In [68]:
def parse_highlights(filename, story_sentences, story_paragraphs, story_sentences_indexes, story_paragraphs_indexes):
  highlight_file = open(f'./highlighting_logs/{filename}.log', 'r')
  CATEGORY_MAP = {'None': 0, 'Present': 1, 'Confused': 2, 'Curious': 3, 'Connected': 4, 'Other': 5}
  lines = highlight_file.readlines()
  highlights = [None] * len(story_paragraphs)
  survey_results = []
  for line in lines:
    start = line.find('Highlight:')
    if start != -1:
      highlight = json.loads(str(line[start + len('Highlight: '):].replace("'", '"')), )
      highlight_absolute_indexes = update_indexes(highlight, story_paragraphs_indexes)
      highlights[highlight[-1]['paragraph']] = highlight_absolute_indexes
    survey = re.search("Survey\d:\s", line)
    if survey != None:
      answers_str = """{0}"""
      answers_str = answers_str.format(str(line[survey.end():]))

      # oh lord.
      # print(re.sub("'}", '"}', re.sub("{'", '{"', re.sub("', ", '", ', re.sub(r"(\: |\, )'", r'\1"', re.sub("(\d)'", r'\1"', re.sub('\\\\', "", re.sub('"', "'", answers_str))))))))
      survey_dict = json.loads(re.sub("'}", '"}', re.sub("{'", '{"', re.sub("', ", '", ', re.sub(r"(\: |\, )'", r'\1"', re.sub("(\d)'", r'\1"', re.sub('\\\\', "", re.sub('"', "'", answers_str))))))), )
      survey_dict['participant_id'] = filename.split('_')[0].lower()
      survey_dict['story'] = filename.split('_')[1].replace('.log', '')
      survey_results.append(survey_dict)
  sentence_highlights = [{'category': 0, 'proportion': 0.0, 'percent_highlighted': 0.0}] * len(story_sentences)
  for i in range(len(highlights)):
    if highlights[i] != None:
      for k in range(len(highlights[i]) - 1):
        start_sent = None
        end_sent = None
        for p in range(len(story_sentences_indexes) - 1):
          if highlights[i][k]['start'] >= story_sentences_indexes[p] and highlights[i][k]['start'] < story_sentences_indexes[p+1]:
            start_sent = p
          if highlights[i][k]['end'] >= story_sentences_indexes[p] and highlights[i][k]['end'] < story_sentences_indexes[p+1]:
            end_sent = p
          if start_sent != None:
            end_index = story_sentences_indexes[p+1] if end_sent == None else highlights[i][k]['end']
            start_index = story_sentences_indexes[p] if story_sentences_indexes[p] > highlights[i][k]['start'] else highlights[i][k]['start']
            proportion = (end_index - start_index) / (story_sentences_indexes[p+1] - story_sentences_indexes[p])
            if proportion > sentence_highlights[p]['proportion']:
              sentence_highlights[p] = {'category': CATEGORY_MAP[highlights[i][k]['tag']], 'proportion': proportion, 'percent_highlighted': sentence_highlights[p]['percent_highlighted'] + proportion}
            else:
              sentence_highlights[p]['percent_highlighted'] = sentence_highlights[p]['percent_highlighted'] + proportion
          if end_sent != None:
            break
  for sentence in sentence_highlights:
    if sentence['proportion'] == 0.0:
      sentence['proportion'] = 1.0
  highlight_df = pd.DataFrame.from_dict(sentence_highlights)
  highlight_df.to_csv(f'./highlights/{filename}.csv')
  first_survey = pd.DataFrame.from_dict([survey_results[0]])
  last_survey = pd.DataFrame.from_dict([survey_results[1]])
  survey_df = pd.concat([first_survey, last_survey], axis=1)
  return survey_df.loc[:,~survey_df.columns.duplicated()].copy()
  

In [69]:
HIGHLIGHT_DIR = "./highlighting_logs/"
survey_df = pd.DataFrame()
for filename in os.listdir(HIGHLIGHT_DIR):
    f = os.path.join(HIGHLIGHT_DIR,filename)
    if os.path.isfile(f):
        filename = f.replace(HIGHLIGHT_DIR, '').replace('.log', '')
        if 'schoolmistress' in filename:
          survey = parse_highlights(filename, schoolmistress_story_sentences, schoolmistress_story_paragraphs, schoolmistress_story_sentences_indexes, schoolmistress_story_paragraphs_indexes)
        elif 'el' in filename:
          survey = parse_highlights(filename, el_story_sentences, el_story_paragraphs, el_story_sentences_indexes, el_story_paragraphs_indexes)
        if len(survey_df) == 0:
          survey_df = survey
        else:
          survey_df = pd.concat([survey_df, survey], ignore_index=True)
survey_df.to_csv('./survey_results.csv')



In [28]:
def normalize_ids(x):
    return str(x).lower() if "id" in str(x).lower() else "id" + str(x)

In [30]:
combined_df = pd.read_csv("results_combined.csv")
combined_df['RECORDING_SESSION_LABEL'] = combined_df['RECORDING_SESSION_LABEL'].apply(normalize_ids)

In [29]:
highlight_percent=pd.read_csv("./results/id10_el.csv", usecols=['RECORDING_SESSION_LABEL', 'Unnamed: 0.1', 'percent_highlighted'])
highlight_percent['story'] = ['EL']*len(highlight_percent)
highlight_percent['RECORDING_SESSION_LABEL'] = highlight_percent['RECORDING_SESSION_LABEL'].apply(normalize_ids)
for filename in os.listdir('./results/'):
    f = os.path.join('./results/',filename)
    if os.path.isfile(f) and f != "./results/id10_el.csv":
        df = pd.read_csv(f, usecols=['RECORDING_SESSION_LABEL', 'Unnamed: 0.1', 'percent_highlighted'])
        if "schoolmistress" in f:
            df['story'] = ['SM']*len(df)
        else:
            df['story'] = ['EL']*len(df)
        highlight_percent['RECORDING_SESSION_LABEL'] = highlight_percent['RECORDING_SESSION_LABEL'].apply(normalize_ids)
        highlight_percent = pd.concat([highlight_percent, df])


highlight_percent = highlight_percent.drop_duplicates()

In [31]:
merged = pd.merge(combined_df, highlight_percent, how='inner', left_on=['RECORDING_SESSION_LABEL', 'story', 'Unnamed..0.1'], right_on=['RECORDING_SESSION_LABEL', 'story', 'Unnamed: 0.1'], copy=False)

In [32]:
merged.to_csv("combined_with_percent_highlighted.csv")